In [16]:
import json
import pandas as pd
import altair as alt
import numpy as np
import altair as alt
from vega_datasets import data
from altair import datum

alt.renderers.enable('notebook')

def systemA():

    # This JSON file contains a mapping from country names to ISO 3166 Country Codes.
    country_json = 'iso_country.json'

    with open(country_json) as f:
        countries = json.load(f)
        country_names = [country['name'] for country in countries]

    def get_country_id(country_name):
        for country in countries:
            if country['name'] == country_name:
                return country['id']
        return None

    def get_year(date_str):
        return int(date_str[-4:])

    # Read the csv file of dataset.
    df = pd.read_csv('Global_Superstore.csv', low_memory=False)

    # Check the country name and corresponding Country Codes.
    df = df[df['Country'].isin(country_names)]
    # df = df.tail(20000)

    # Generate the "id" and "year" attribute
    df['id'] = df['Country'].apply(get_country_id)
    df['year'] = df['Order Date'].apply(get_year)

    # Convert the 'Quantity' column to numeric data type
    df['Quantity'] = df['Quantity'].astype(float)

    #--------------------

    dfA = df

    # Group "id", "year" and "Category", calculate the sum of "Quantity" and save the result to a new dataframe B
    dfB = dfA.groupby(["id", "year", "Country", "Category"]).agg({"Quantity": "sum"}).reset_index()

    # Rename the "Quantity" column to "part_count"
    dfB = dfB.rename(columns={"Quantity": "part_count"})

    # Use "id" and "year" for grouping, calculate the sum of "count" and assign the result to the new column "sum_count"
    dfB["count"] = dfB.groupby(["id", "year"])["part_count"].transform("sum")

    grouped_df = dfB

    #---------------------------
    countries = alt.topo_feature(data.world_110m.url, 'countries')

    slider = alt.binding_range(
        name="year",
        step=1,
        min=df['year'].min(), 
        max=df['year'].max()
    )

    select_date = alt.selection_single(
        name="year", 
        fields=['year'],
        bind=slider, 
    )

    # Data generators for the background
    sphere = alt.sphere()
    graticule = alt.graticule()

    # Source of land data
    source = alt.topo_feature(data.world_110m.url, 'countries')

    myChart = alt.Chart(grouped_df)

    chart1 = alt.layer(
        alt.Chart(source).mark_geoshape(
        fill='#666666',
        stroke='white'
    ),
        myChart.mark_geoshape()\
        .encode(color=alt.Color('count:Q',scale=alt.Scale(type='log', scheme='magma', nice=True), legend=alt.Legend(titleOrient='left')), tooltip=['Country', 'count'])\
        .add_selection(select_date)\
        .transform_filter(select_date)\
        .transform_lookup(
            lookup='id',
            from_=alt.LookupData(countries, key='id',
                                 fields=["type", "properties", "geometry"])
        ))\
        .project('equirectangular')\
        .properties(
            width=700,
            height=400,
            title='Quantity of Online Order from Different Countries'
        )

    chart2 = myChart.add_selection(select_date).mark_bar().encode(
        alt.X('sum(part_count):Q', axis=alt.Axis(title='count')),
        alt.Y('Country:N', sort=alt.EncodingSortField(field="count", order='descending')),
        alt.Color('Category:N',scale=alt.Scale(type='log', scheme='set2', nice=True), legend=alt.Legend(titleOrient='left')),
        tooltip=['Country', 'part_count']
    )\
    .transform_filter(select_date).transform_window(
        rank='rank(count)',
        sort=[alt.SortField('count', order='descending')],

    ).transform_filter(
        alt.datum.rank <= 30
    ).properties(width=700, height=200, title='Top 10 Countries in Online Order Quantities for the Current Year')

    alt.vconcat(chart1, chart2).resolve_scale(
        color='independent'
    )

    # Create a data frame with very few data
    empty_data = pd.DataFrame({'x': [0], 'y': [0]})

    # Create a blank chart using blank data
    blank_chart = alt.Chart(empty_data).mark_bar().encode(
        x=alt.X('x:Q', title='', axis=None),
        y=alt.Y('y:Q', title='', axis=None),
        color=alt.value('white')
    ).properties(
        title='',
        width=30,
        height=10
    )

    down_chart = alt.hconcat((blank_chart | chart2), blank_chart).resolve_scale(
        color='independent'
    )

    up_chart = alt.hconcat((blank_chart | chart1), blank_chart).resolve_scale(
        color='independent'
    )


    chart = alt.vconcat(up_chart, down_chart).configure_view(stroke='white')
    chart.save('SystemA.html')
    chart.show()
    
# import altair as alt
# alt.renderers.enable('notebook')
systemA()

Displaying chart at http://localhost:55187/
